In [7]:
import os, random
import numpy as np
import matplotlib.pyplot as plt
from owslib.wms import WebMapService   
from owslib.wfs import WebFeatureService

if os.getcwd().endswith('bomenspotter/notebooks') or os.getcwd().endswith('bomenspotter\\notebooks'):
    os.chdir('../src/data/datatools')
elif os.getcwd().endswith('bomenspotter/notebooks/Habitat_typen'):
    os.chdir('../../src/data/datatools')
    
from collector import Collector, CollectorConfig
from validation import  TOP10NL_Validator, Aerial_Validator, Ticket
from TOP10NL import TOP10NL_SIMPLE
import random, json, secrets, collections

if not os.getcwd().endswith('bomenspotter'):
    os.chdir('../../../')    
    
work_directory = os.getcwd()

In [9]:
from pathlib import Path
home = str(Path.home())

In [10]:
home

'/home/jovyan'

In [2]:
def ConversionTabelMaand(maand_numeric):
    tabel = {'Maand_numeriek':['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
            'Maand':['januari', 'februari', 'maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december']}
    tabel_df = pd.DataFrame(data=tabel)
    maand_result = tabel_df[tabel_df['Maand_numeriek'] == maand_numeric]
    result = maand_result["Maand"].values[0]
    return(result)    


In [3]:
import requests
import json
url = 'https://www.satellietdataportaal.nl/resFilter.php'
payload = {"sensor":"SuperView-1","resolution":"0.5"}
header = {'content-type': 'application/x-www-form-urlencoded; charset=UTF-8'}
data = json.dumps(payload).encode('utf-8')
r = requests.get(url, data = data, headers = header)


In [4]:
#https://stackoverflow.com/questions/15900338/python-request-post-with-param-data
import requests
import json

# URL voor het zoeken van data op het satelliet dataportaal
url = 'https://www.satellietdataportaal.nl/searchSatData.php'
headers = {'content-type': 'application/x-www-form-urlencoded; charset=UTF-8'}
# Parameters op basis van een coordinaat (lat, lon)
params = 'searchGeoJson%5Btype%5D=Feature&searchGeoJson%5Bgeometry%5D%5Btype%5D=Point&searchGeoJson%5Bgeometry%5D%5Bcoordinates%5D%5B%5D=5.963379&searchGeoJson%5Bgeometry%5D%5Bcoordinates%5D%5B%5D=52.042862'
# Params as json: check https://github.com/github/fetch/issues/263
response = requests.post(url,  data=params, headers = headers)
#t.content
#response.text
response_string = response.text
response_json = json.loads(response_string)
results = response_json.get('Results')

In [ ]:

import sys
sys.path.insert(0, '/home/jovyan/bomenspotter/src/data/datatools')
import ogr2ogr
import pandas as pd
from shapely.geometry import mapping, Polygon
import fiona

productnaam = 'Tri_RD_12bit_RGBI_80cm'
gebruikersnaam = 'baswiltenburg'
wachtwoord = 'Maandag1!'
jaar = '2018'
maand = 'juli'
treshold_cloudcover_scene = 0.1
treshold_cloudcover = 0.2

# Loop trough the request results and save only the nested dictionaries when 'sensorname' is the desired sensor (harcoded on 'TripleSat')
data_list_desired_sensor = []
for dic in results:
    sensor = dic.get('sensorname')
    if sensor == 'TripleSat':
        #print(dic['acqtime'])
        #print(dic['cloudcover_scene'])
        #print(dic['cloudy'])
        data_list_desired_sensor.append(dic)

# Loop trough the data of the desired sensor and filter on year and month 
data_list_query = []
for dic in data_list_desired_sensor:
    acqtime = dic.get('acqtime')
    acqjaar = acqtime[0:4]
    acqmaand = ConversionTabelMaand(str(acqtime[5:7]))
    if acqjaar == jaar and acqmaand == maand:
        data_list_query.append(dic)
        
#print(data_list_query)

data_list_query2 = []
for dic in data_list_query:
    if (dic['cloudcover_scene'] <= treshold_cloudcover_scene and dic['cloudy'] <= treshold_cloudcover):
        data_list_query2.append(dic)
        
#print(data_list_query2)
for sat_photo in data_list_query2:
    layer_link = sat_photo['layer_link']
    #layer = layer_link.split('/')[6][0:19]
    layer = layer_link.split('/')[6].split('_')
    layer = layer[0]+'_'+layer[1]+'_'+layer[2]
    send_layer = '/Download/'+layer

    url2 = 'https://www.satellietdataportaal.nl/login.php'
    #params2 = {'password':wachtwoord, 'refLink':send_layer, 'username':gebruikersnaam}
    params2 = {'password':'Maandag1!', 'refLink':send_layer, 'username':'baswiltenburg'}
    headers2 = {'content-type': 'application/x-www-form-urlencoded; charset=UTF-8'}
    download_response = requests.post(url2,  data=params2, headers = headers2)
    download_json = json.loads(download_response.text)
    product_data = download_json['data']['products'] #List of products       

    for product in product_data:
        if product['productname'] == productnaam:                     
            link = product['link']
            #print(link)
            #req = requests.get(link, auth=('baswiltenburg', 'Maandag1!'))
            header = {'Content-Type':'text/html; charset=UTF-8'}
            req = requests.get(link, stream = False, headers= header)
            print(req.content)



        
#https://pybit.es/requests-session.html

            
# Create polygon to write 
#dicnew = json.loads(image_dic['geojson'])
#coordinates = dicnew['coordinates'][0]
#poly = Polygon(coordinates)

# Reference link for downloading: /Download/20180712_092441_Tri
# response = 20180712_092444_Tri

# https://www.satellietdataportaal.nl/Download/20180629_093714_Tri/Tri_RD_12bit_RGBI_80cm
# https://www.satellietdataportaal.nl/Download/20180712_092444_Tri/Tri_RD_12bit_RGBI_80cm
    

   
    

In [5]:
# TEST 3
import sys
sys.path.insert(0, '/home/jovyan/bomenspotter/src/data/datatools')
import ogr2ogr
import pandas as pd
from shapely.geometry import mapping, Polygon
import fiona
from requests import Request, Session


productnaam = 'Tri_RD_12bit_RGBI_80cm'
gebruikersnaam = 'baswiltenburg'
wachtwoord = 'Maandag1!'
jaar = '2018'
maand = 'september'
treshold_cloudcover_scene = 0.1
treshold_cloudcover = 0.2

In [6]:
from requests import session

s = session()
print(s.cookies)

# Loop trough the request results and save only the nested dictionaries when 'sensorname' is the desired sensor (harcoded on 'TripleSat')
data_list_desired_sensor = []
for dic in results:
    sensor = dic.get('sensorname')
    if sensor == 'TripleSat':
        #print(dic['acqtime'])
        #print(dic['cloudcover_scene'])
        #print(dic['cloudy'])
        data_list_desired_sensor.append(dic)

# Loop trough the data of the desired sensor and filter on year and month 
data_list_query = []
for dic in data_list_desired_sensor:
    acqtime = dic.get('acqtime')
    acqjaar = acqtime[0:4]
    acqmaand = ConversionTabelMaand(str(acqtime[5:7]))
    if acqjaar == jaar and acqmaand == maand:
        data_list_query.append(dic)
        


data_list_query2 = []
for dic in data_list_query:
    if (dic['cloudcover_scene'] <= treshold_cloudcover_scene and dic['cloudy'] <= treshold_cloudcover):
        data_list_query2.append(dic)
        
#print(data_list_query2) # ALS LEEG: GEEN BEELDEN

for sat_photo in data_list_query2:
    layer_link = sat_photo['layer_link']
    #layer = layer_link.split('/')[6][0:19]
    layer = layer_link.split('/')[6].split('_')
    layer = layer[0]+'_'+layer[1]+'_'+layer[2]
    send_layer = '/Download/'+layer
    #print(send_layer)
    url2 = 'https://www.satellietdataportaal.nl/login.php'
    #params2 = {'password':wachtwoord, 'refLink':send_layer, 'username':gebruikersnaam}
    params2 = {'password':'Maandag1!', 'refLink':send_layer, 'username':'baswiltenburg'}
    headers2 = {'content-type': 'application/x-www-form-urlencoded; charset=UTF-8'}     
    download_response = s.post(url2, data=params2, headers = headers2)
    cookie = s.cookies
    print(cookie)
    #download_response = session.post(url2,  data=params2, headers = headers2, stream = False)
    download_json = json.loads(download_response.text)
    product_data = download_json['data']['products'] #List of products 
    for product in product_data:
        if product['productname'] == productnaam:  
            link = product['link']
            print(link)
            header = {'Content-Type':'text/html; charset=UTF-8'}
            {'Accept': 'application/json, text/javascript, */*; q=0.01',
             'Accept-Encoding': 'gzip, deflate, br',
             'Accept-Language': 'en-US,en;q=0.9',
             'Connection': 'keep-alive',
             'Host': 'www.satellietdataportaal.nl',
             'Referer': 'https://www.satellietdataportaal.nl/',
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
             'X-Requested-With': 'XMLHttpRequest'}      
            req = s.get(link, stream = False, headers= header)
            print(req)
            print(type(req.content))
            with open(work_directory+"/data/test/DownloadTest2.zip", "wb") as file:
                file.write(req.content)   
                              
# https://pybit.es/requests-session.html
# http://docs.python-requests.org/en/master/user/advanced/
            
# Create polygon to write 
#dicnew = json.loads(image_dic['geojson'])
#coordinates = dicnew['coordinates'][0]
#poly = Polygon(coordinates)

# Reference link for downloading: /Download/20180712_092441_Tri
# response = 20180712_092444_Tri

# https://www.satellietdataportaal.nl/Download/20180629_093714_Tri/Tri_RD_12bit_RGBI_80cm
# https://www.satellietdataportaal.nl/Download/20180712_092444_Tri/Tri_RD_12bit_RGBI_80cm

# print(type(cookie))
   
    

<RequestsCookieJar[]>
<RequestsCookieJar[<Cookie PHPSESSID=upmkf73sdbeiaeaiglk63ttqkg for www.satellietdataportaal.nl/>]>
https://www.satellietdataportaal.nl/Download/20180918_094019_Tri/Tri_RD_12bit_RGBI_80cm
<Response [200]>
<class 'bytes'>


In [ ]:
# inloggen
url = 'https://www.satellietdataportaal.nl/login.php'
params = {'password':'Maandag1!', 'refLink':'/Download/20190318_104441_SV1-01', 'username':'baswiltenburg'}
headers = {'content-type': 'application/x-www-form-urlencoded; charset=UTF-8'}
t = requests.post(url,  data=params, headers = headers)

In [ ]:
# DONWLOAD VAN WMS #
url= "https://satellietdataportaal.nl/wmslayer/MOS_Triplesat_juni_2018"
veluwe = [174500, 400000, 175000, 400500]

wms = WebMapService(url = 'https://satellietdataportaal.nl/wmslayer/MOS_Triplesat_september_2018', version= '1.3.0')

img = wms.getmap(layers=['Triplesat_september_2018'], styles=[], srs='EPSG:28992', crs='EPSG:28992', bbox=veluwe, 
                 size=(100,100), format='image/jpeg', transparent = True) 

filename = work_directory + "/data/test/" + "test.jpeg"  
out = open(filename, 'wb')
out.write(img.read())
out.close()  